In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import sys
import os
sys.path.append('..')

import torch
import optuna
import pytorch_lightning as pl

from radianet import get_dataloader
from radianet.datasets import SampleDataset2D, LIDCDataset, Transforms
from radianet.config import Config
from radianet.models import Simple3DCNN, MyEfficientNet
from radianet.callbacks import MetricsCallback
from radianet.lightningnet import LightningNet


In [3]:


def objective(trial):
    
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        os.path.join(Config.MODEL_DIR, "trial_{}".format(trial.number), "{epoch}"), 
        monitor="val_loss", save_top_k = -1
    )
    
    metrics_callback = MetricsCallback()

    trainer = pl.Trainer(
        logger=False,
        max_epochs=Config.EPOCHS,
        checkpoint_callback=checkpoint_callback,
        callbacks=[metrics_callback],
        gpus=0 if torch.cuda.is_available() else None
    )
    
    transforms = Transforms()
    #datasets = LIDCDataset(transforms, Config.LIDC_PATH)
    datasets = SampleDataset2D(transforms, n_sample = 10)
    dataloader = get_dataloader(datasets, Config.BATCHSIZE)

    lightning_model = LightningNet(trial, Config, MyEfficientNet, dataloader)
    print("before git")
    trainer.fit(lightning_model)
    
    print("here")
    return metrics_callback.metrics[-1]["val_loss"]

study = optuna.create_study(study_name = Config.NAME, storage = Config.DB_NAME, load_if_exists = True)
study.optimize(objective, n_trials= Config.N_TRIALS)


[I 2020-05-12 13:04:06,734] A new study created with name: test_chechpoint
INFO:lightning:GPU available: True, used: False


Loaded pretrained weights for efficientnet-b0
before git
linear input not have been calculate


INFO:lightning:
    | Name                                                 | Type                    | Params
---------------------------------------------------------------------------------------------
0   | model                                                | MyEfficientNet          | 5 M   
1   | model.head                                           | Sequential              | 928   
2   | model.head.0                                         | Conv2dStaticSamePadding | 864   
3   | model.head.0.static_padding                          | ZeroPad2d               | 0     
4   | model.head.1                                         | BatchNorm2d             | 64    
5   | model.body_no_grad                                   | Sequential              | 2 M   
6   | model.body_no_grad.0                                 | MBConvBlock             | 1 K   
7   | model.body_no_grad.0._depthwise_conv                 | Conv2dStaticSamePadding | 288   
8   | model.body_no_grad.0._depthwise_conv.s

torch.Size([2, 1])


torch.Size([2, 1])


..\radianet\lightningnet.py:49: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])) is deprecated. Please ensure they have the same size.
  return {"loss": F.binary_cross_entropy(output, target)}


torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([2, 1])


torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([2, 1])


torch.Size([2, 1])

here


[I 2020-05-12 13:04:22,788] Finished trial#0 with value: 0.3460502028465271 with parameters: {'lr': 0.0005505348325043929}. Best is trial#0 with value: 0.3460502028465271.


In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))